# Analyze startup investments from Crunchbase.com

Given the large dataset with over 50,000 rows, it's essential to read it into dataframes using 5,000 row chunks to ensure that each chunk consumes less than 10 megabytes of memory.

Across all the chunks, the following steps should be taken:

1. Analyze each column's missing value counts.
2. Assess each column's memory footprint.
3. Calculate the total memory footprint of all the chunks combined.
4. Identify columns that can be dropped because they are not useful for analysis.

In [1]:
import pandas as pd
pd.options.display.max_columns = 99
chunk_iter = pd.read_csv('crunchbase-investments.csv', chunksize=5000, encoding='ISO-8859-1')

In [2]:
mv_list = []
for chunk in chunk_iter:
    mv_list.append(chunk.isnull().sum())
    
combined_mv_vc = pd.concat(mv_list)
unique_combined_mv_vc = combined_mv_vc.groupby(combined_mv_vc.index).sum()
unique_combined_mv_vc.sort_values()

company_country_code          1
company_name                  1
company_permalink             1
company_region                1
investor_region               2
investor_permalink            2
investor_name                 2
funded_quarter                3
funded_at                     3
funded_month                  3
funded_year                   3
funding_round_type            3
company_state_code          492
company_city                533
company_category_code       643
raised_amount_usd          3599
investor_country_code     12001
investor_city             12480
investor_state_code       16809
investor_category_code    50427
dtype: int64

In [3]:
chunk_iter = pd.read_csv('crunchbase-investments.csv', chunksize=5000, encoding='ISO-8859-1')
counter = 0
series_memory_fp = pd.Series(dtype='float64')
for chunk in chunk_iter:
    if counter == 0:
        series_memory_fp = chunk.memory_usage(deep=True)
    else:
        series_memory_fp += chunk.memory_usage(deep=True)
    counter += 1

# Drop memory footprint calculation for the index.
series_memory_fp = series_memory_fp.drop('Index')
series_memory_fp

company_permalink         4057788
company_name              3591326
company_category_code     3421104
company_country_code      3172176
company_state_code        3106051
company_region            3411545
company_city              3505886
investor_permalink        4980548
investor_name             3915666
investor_category_code     622424
investor_country_code     2647292
investor_state_code       2476607
investor_region           3396281
investor_city             2885083
funding_round_type        3410707
funded_at                 3542185
funded_month              3383584
funded_quarter            3383584
funded_year                422960
raised_amount_usd          422960
dtype: int64

In [4]:
series_memory_fp.sum() / (1024 * 1024)

56.98753070831299

In [5]:
unique_combined_mv_vc.sort_values()

company_country_code          1
company_name                  1
company_permalink             1
company_region                1
investor_region               2
investor_permalink            2
investor_name                 2
funded_quarter                3
funded_at                     3
funded_month                  3
funded_year                   3
funding_round_type            3
company_state_code          492
company_city                533
company_category_code       643
raised_amount_usd          3599
investor_country_code     12001
investor_city             12480
investor_state_code       16809
investor_category_code    50427
dtype: int64

In [6]:
# Drop columns representing URLs or containing too many missing values (>90% missing)
drop_cols = ['investor_permalink', 'company_permalink', 'investor_category_code']
keep_cols = chunk.columns.drop(drop_cols)

In [7]:
keep_cols.tolist

<bound method IndexOpsMixin.tolist of Index(['company_name', 'company_category_code', 'company_country_code',
       'company_state_code', 'company_region', 'company_city', 'investor_name',
       'investor_country_code', 'investor_state_code', 'investor_region',
       'investor_city', 'funding_round_type', 'funded_at', 'funded_month',
       'funded_quarter', 'funded_year', 'raised_amount_usd'],
      dtype='object')>

In [8]:
# Key: Column name, Value: List of types
col_types = {}
chunk_iter = pd.read_csv('crunchbase-investments.csv', chunksize=5000, encoding='ISO-8859-1', usecols=keep_cols)

for chunk in chunk_iter:
    for col in chunk.columns:
        if col not in col_types:
            col_types[col] = [str(chunk.dtypes[col])]
        else:
            col_types[col].append(str(chunk.dtypes[col]))

In [9]:
uniq_col_types = {}
for k,v in col_types.items():
    uniq_col_types[k] = set(col_types[k])
uniq_col_types

{'company_name': {'object'},
 'company_category_code': {'object'},
 'company_country_code': {'object'},
 'company_state_code': {'object'},
 'company_region': {'object'},
 'company_city': {'object'},
 'investor_name': {'object'},
 'investor_country_code': {'float64', 'object'},
 'investor_state_code': {'float64', 'object'},
 'investor_region': {'object'},
 'investor_city': {'float64', 'object'},
 'funding_round_type': {'object'},
 'funded_at': {'object'},
 'funded_month': {'object'},
 'funded_quarter': {'object'},
 'funded_year': {'float64', 'int64'},
 'raised_amount_usd': {'float64'}}

In [10]:
chunk

,company_name,company_category_code,company_country_code,company_state_code,company_region,company_city,investor_name,investor_country_code,investor_state_code,investor_region,investor_city,funding_round_type,funded_at,funded_month,funded_quarter,funded_year,raised_amount_usd
50000,NuORDER,fashion,USA,CA,Los Angeles,West Hollywood,Mortimer Singer,NaN,NaN,unknown,NaN,series-a,2012-10-01,2012-10,2012-Q4,2012,3060000.0
50001,ChaCha,advertising,USA,IN,Indianapolis,Carmel,Morton Meyerson,NaN,NaN,unknown,NaN,series-b,2007-10-01,2007-10,2007-Q4,2007,12000000.0
50002,Binfire,software,USA,FL,Bocat Raton,Bocat Raton,Moshe Ariel,NaN,NaN,unknown,NaN,angel,2008-04-18,2008-04,2008-Q2,2008,500000.0
50003,Binfire,software,USA,FL,Bocat Raton,Bocat Raton,Moshe Ariel,NaN,NaN,unknown,NaN,angel,2010-01-01,2010-01,2010-Q1,2010,750000.0
50004,Unified Color,software,USA,CA,SF Bay,South San Frnacisco,Mr. Andrew Oung,NaN,NaN,unknown,NaN,angel,2010-01-01,2010-01,2010-Q1,2010,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52865,Garantia Data,enterprise,USA,CA,SF Bay,Santa Clara,Zohar Gilon,NaN,NaN,unknown,NaN,series-a,2012-08-08,2012-08,2012-Q3,2012,3800000.0
52866,DudaMobile,mobile,USA,CA,SF Bay,Palo Alto,Zohar Gilon,NaN,NaN,unknown,NaN,series-c+,2013-04-08,2013-04,2013-Q2,2013,10300000.0
52867,SiteBrains,software,USA,CA,SF Bay,San Francisco,zohar israel,NaN,NaN,unknown,NaN,angel,2010-08-01,2010-08,2010-Q3,2010,350000.0
52868,Comprehend Systems,enterprise,USA,CA,SF Bay,Palo Alto,Zorba Lieberman,NaN,NaN,unknown,NaN,series-a,2013-07-11,2013-07,2013-Q3,2013,8400000.0


# Loading Chunks into SQLite

Create and connect to a new SQLite database file.

Expand on the existing chunk processing code to export each chunk to a new table in the SQLite database.

In [23]:
import sqlite3
conn = sqlite3.connect('crunchbase.db')
chunk_iter = pd.read_csv('crunchbase-investments.csv', chunksize=5000, encoding='ISO-8859-1')

for chunk in chunk_iter:
    chunk.to_sql("investments", conn, if_exists='append', index=False)
    

Use the pandas SQLite workflow to answer the below:

1. What proportion of the total amount of funds did the top 10% raise?
2. Which category of company attracted the most investments?
3. Which investor contributed the most money (across all startups)?
4. Which investors contributed the most money per startup?
5. Which funding round was the most popular? Which was the least popular?

In [32]:
query = 'SELECT SUM(raised_amount_usd)/(SELECT SUM(raised_amount_usd) FROM investments) AS raised_by_top10 FROM investments GROUP BY company_name ORDER BY raised_by_top10 DESC LIMIT 10;'
results = pd.read_sql_query(query, conn)

In [30]:
results

,rasied_by_top10
0,0.043536
1,0.014940
2,0.006608
3,0.006093
4,0.004767
5,0.004233
6,0.004090
7,0.004055
8,0.003938
9,0.003814


In [43]:
query1 = 'SELECT company_category_code, SUM(raised_amount_usd) AS category_most_investment_raised FROM investments GROUP BY company_name ORDER BY SUM(raised_amount_usd) DESC LIMIT 1'
results1 = pd.read_sql_query(query1, conn)

In [44]:
results1

,company_category_code,category_most_investment_raised
0,mobile,1.187200e+11


In [59]:
query2 = 'SELECT investor_name, SUM(raised_amount_usd) AS investor_most_investment_raised FROM investments GROUP BY investor_name ORDER BY SUM(raised_amount_usd) DESC LIMIT 1'
results2 = pd.read_sql_query(query2, conn)

In [60]:
results2

,investor_name,investor_most_investment_raised
0,Kleiner Perkins Caufield & Byers,4.487131e+10


In [57]:
query3 = 'SELECT investor_name, SUM(raised_amount_usd) AS investor_most_investment_raised_per_startup FROM investments GROUP BY investor_name ORDER BY SUM(raised_amount_usd) DESC'
results3 = pd.read_sql_query(query3, conn)

In [58]:
results3

,investor_name,investor_most_investment_raised_per_startup
0,Kleiner Perkins Caufield & Byers,4.487131e+10
1,New Enterprise Associates,3.877017e+10
2,Accel Partners,2.588850e+10
3,Goldman Sachs,2.550184e+10
4,Sequoia Capital,2.415761e+10
...,...,...
10461,313 Ventures,NaN
10462,2x Consumer Products Growth Partners,NaN
10463,212 Capital Partners,NaN
10464,1in10 Ventures,NaN


In [61]:
query4 = 'SELECT funding_round_type, SUM(raised_amount_usd) AS investment_raised FROM investments GROUP BY funding_round_type ORDER BY SUM(raised_amount_usd) DESC LIMIT 1'
results4 = pd.read_sql_query(query4, conn)

In [62]:
results4

,funding_round_type,investment_raised
0,series-c+,1.063014e+12


In [63]:
query5 = 'SELECT funding_round_type, SUM(raised_amount_usd) AS investment_raised FROM investments GROUP BY funding_round_type HAVING SUM(raised_amount_usd)>0 ORDER BY SUM(raised_amount_usd) LIMIT 1'
results5 = pd.read_sql_query(query5, conn)

In [64]:
results5

,funding_round_type,investment_raised
0,crowdfunding,25966000.0
